In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from joblib import dump, load
from pathlib import Path

In [ ]:
flats_data = pd.read_excel('data/flats.xlsx')
euro_data = pd.read_excel('data/euro.xlsx')
flats_data['renovation'] = 0
euro_data['renovation'] = 1
merged_data = pd.merge(flats_data, euro_data, how='outer')

In [ ]:
flats_data['house_wall_type'] = flats_data['house_wall_type'].fillna('Неизвестно')
merged_data['is_apartment'] = merged_data['is_apartment'].fillna(0)
merged_data['house_wall_type'] = merged_data['house_wall_type'].fillna('Неизвестно')
merged_data['all_data.object_info.living_area'] = merged_data['all_data.object_info.living_area'].fillna(
    merged_data["all_data.object_info.living_area"].mean())
merged_data['build_year'] = merged_data['build_year'].fillna(merged_data['build_year'].mean())

merged_data.dropna(subset=['price_sq', 'area'], inplace=True)

merged_data['city'], cities = pd.factorize(merged_data['city'])
merged_data['house_wall_type'], wall_types = pd.factorize(merged_data['house_wall_type'])

In [ ]:
X = merged_data[['city', 'lat', 'lon', 'area', 'rooms', 'floor', 'house_floors',
                 'renovation', 'build_year', 'house_wall_type', 'kitchen_area',
                 'all_data.object_info.living_area', 'is_apartment', ]]
y = merged_data['price_sq']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
dump(model, 'models/model_linear.pkl')

model = DecisionTreeRegressor()
model.fit(X_train, y_train)
dump(model, 'models/model_decision_tree.pkl')

model = RandomForestRegressor()
model.fit(X_train, y_train)
dump(model, 'models/model_random_forest.pkl')

model = GradientBoostingRegressor()
model.fit(X_train, y_train)
dump(model, 'models/model_gradient_boosting.pkl')

In [6]:
# Metrics
for file in Path("models").glob('model*'):
    model = load(file)
    y_pred = model.predict(X_test)
    r_mse = root_mean_squared_error(y_pred, y_test)
    mae = mean_absolute_error(y_pred, y_test)
    r2 = r2_score(y_pred, y_test)
    print(f"{file.name} \nRMSE: {round(r_mse)} MAE: {round(mae)}, R2: {r2:.4}\n")

model_decision_tree.pkl 
RMSE: 34731 MAE: 21637, R2: 0.8376

model_gradient_boosting.pkl 
RMSE: 31103 MAE: 21605, R2: 0.8325

model_linear.pkl 
RMSE: 44381 MAE: 31273, R2: 0.6135

model_random_forest.pkl 
RMSE: 24838 MAE: 16527, R2: 0.9041


In [7]:
model = load('models/model_random_forest.pkl')  # Best
flats_data = pd.read_excel('data/flats.xlsx')
flats_data['house_wall_type'] = flats_data['house_wall_type'].fillna('Неизвестно')


def prepare_data(row):
    data = pd.DataFrame({'city': [cities.get_loc(row['city'])],
                         'lat': [row['lat']],
                         'lon': [row['lon']],
                         'area': [row['area']],
                         'rooms': [row['rooms']],
                         'floor': [row['floor']],
                         'house_floors': [row['house_floors']],
                         'renovation': [1],
                         'build_year': [row['build_year']],
                         'house_wall_type': [wall_types.get_loc(row['house_wall_type'])],
                         'kitchen_area': [row['kitchen_area']],
                         'all_data.object_info.living_area': [row['all_data.object_info.living_area']],
                         'is_apartment': [row['is_apartment']]
                         })

    return model.predict(data), data


def euro_price(row):
    predict, data = prepare_data(row)
    try:
        renovation_value = round(y_pred[0] * data['area'][0])
    except ValueError:
        renovation_value = round(y_pred[0])
    return renovation_value


def euro_sq_price(row):
    return round(prepare_data(row)[0][0])


flats_data['euro_price'] = flats_data.apply(euro_price, axis=1)
flats_data['euro_sq_price'] = flats_data.apply(euro_sq_price, axis=1)
flats_data.to_excel('flats.xlsx', index=False)